# Лекция 2.1: Линейная и полиномиальная регрессия

## Регрессия

**Регрессия** – это метод машинного обучения, который используется для предсказания числового значения на основе входных данных.

> Тут будут рассмотрены, в основном, линейные модели, подробнее изучить которые можно в соответствующей главе [учебника](https://education.yandex.ru/handbook/ml/article/linear-models)

### Линейная регрессия

#### Постановка задачи
Пусть имеется обучающая выборка $ \mathcal{D} = \{(\mathbf{x}_i, y_i)\}_{i=1}^{N} $, где
- $ \mathbf{x}_i \in \mathbb{R}^d $ – вектор признаков,
- $ y_i \in \mathbb{R} $ – непрерывное значение.

Модель линейной регрессии принимает вид:
$$
f(\mathbf{x}) = \mathbf{w}^\top \mathbf{x} + b,
$$
где $ \mathbf{w} \in \mathbb{R}^d $ – вектор весов, а $ b \in \mathbb{R} $ – смещение (также можно включить его в вектор, добавив столбец единиц в матрицу признаков).

#### Критерий оптимальности
Классическим выбором функции потерь (функции, позволяющей оценивать ошибку) является **сумма квадратов ошибок**:
$$
\mathcal{L}(\mathbf{w}, b) = \sum_{i = 1}^N \left( y_i - (\mathbf{w}^\top \mathbf{x}_i + b) \right)^2.
$$
С целью удобства можно записать задачу в матричном виде. Обозначим:
- $ \mathbf{y} = [y_1, y_2, \dots, y_N]^\top $,
- $ X = \begin{pmatrix}
\mathbf{x}_1^\top \\
\mathbf{x}_2^\top \\
\vdots \\
\mathbf{x}_N^\top
\end{pmatrix} \in \mathbb{R}^{N \times d} $.

При включении смещения в вектор признаков, формируем расширенную матрицу:
$$
\tilde{X} = \begin{pmatrix}
1 & \mathbf{x}_1^\top \\
1 & \mathbf{x}_2^\top \\
\vdots & \vdots \\
1 & \mathbf{x}_N^\top
\end{pmatrix} \in \mathbb{R}^{N \times (d+1)},
$$
а параметрический вектор:
$$
\tilde{\mathbf{w}} = \begin{pmatrix} b \\ \mathbf{w} \end{pmatrix} \in \mathbb{R}^{d+1}.
$$
Тогда функция потерь представляется как:
$$
\mathcal{L}(\tilde{\mathbf{w}}) = \|\mathbf{y} - \tilde{X} \tilde{\mathbf{w}}\|^2.
$$

#### Аналитическое решение
Задача минимизации является квадратичной, и условие оптимальности задается равенством нулю градиента:
$$
\nabla_{\tilde{\mathbf{w}}} \mathcal{L}(\tilde{\mathbf{w}}) = -2 \tilde{X}^\top (\mathbf{y} - \tilde{X} \tilde{\mathbf{w}}) = \mathbf{0}.
$$
Отсюда получаем уравнение:
$$
\tilde{X}^\top \tilde{X} \tilde{\mathbf{w}} = \tilde{X}^\top \mathbf{y}.
$$
При предположении обратимости матрицы $ \tilde{X}^\top \tilde{X} $ аналитическое решение имеет вид:
$$
\tilde{\mathbf{w}} = (\tilde{X}^\top \tilde{X})^{-1} \tilde{X}^\top \mathbf{y}.
$$

Это так называемое точное решение задачи регрессии, оно точное, но вычислительно неэффективное, поскольку обращение матрицы обладает сложностью $\mathcal{O}(n^3)$ и, хотя и может быть ускорено, численно нестабильно при мультиколлинеарности или в случае, когда $ \tilde{X}^\top \tilde{X} $ вырождена.

<img src="../../images/regression.png" alt="Регрессия" width="600">

Практический пример:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)
N = 100
x = 10 * np.random.rand(N, 1)

b_true = 5.0
w_true = 2.0

noise_std = 1.0
y = b_true + w_true * x + np.random.randn(N, 1) * noise_std

In [ ]:
X_extended = np.hstack((np.ones((N, 1)), x))

XtX = X_extended.T @ X_extended
XtY = X_extended.T @ y
w_hat = np.linalg.inv(XtX) @ XtY

print("Solution:")
print("Bias (b): {:.4f}".format(w_hat[0, 0]))
print("Weight (w): {:.4f}".format(w_hat[1, 0]))

In [ ]:
y_pred = X_extended @ w_hat

plt.figure(figsize=(8, 6))
plt.scatter(x, y, color="blue", label="Data")

sort_idx = np.argsort(x[:, 0])
x_sorted = x[sort_idx]
X_sorted = np.hstack((np.ones((N, 1)), x_sorted))
y_line = X_sorted @ w_hat

plt.plot(x_sorted, y_line, color="red", linewidth=2, label="Linear Regression")
plt.xlabel("Feature (x)")
plt.ylabel("Target (y)")
plt.title("Linear Regression")
plt.legend()
plt.grid(True)
plt.show()

### Полиномиальная регрессия

#### Постановка задачи
Полиномиальная регрессия – это частный случай нелинейной регрессии, где исходная зависимость между признаками и целевой переменной моделируется полиномом. Пусть имеется одномерный признак $ x $. Модель полиномиальной регрессии степени $ p $ имеет вид:
$$
f(x) = \sum_{j=0}^{p} w_j x^j.
$$
Обратите внимание, что при введении нового векторного представления признаков
$$
\phi(x) = \begin{pmatrix} 1 \\ x \\ x^2 \\ \vdots \\ x^p \end{pmatrix},
$$
модель записывается в виде линейной зависимости от параметров:
$$
f(x) = \mathbf{w}^\top \phi(x),
$$
где $ \mathbf{w} = (w_0, w_1, \dots, w_p)^\top $.

Для многомерного случая применяется аналогичное преобразование, где каждое первоначальное наблюдение расширяется набором полиномиальных признаков (иногда с перекрёстными произведениями).

#### Аналитическое решение
Если обозначить преобразование данных через матрицу:
$$
\Phi = \begin{pmatrix}
\phi(x_1)^\top \\
\phi(x_2)^\top \\
\vdots \\
\phi(x_N)^\top
\end{pmatrix} \in \mathbb{R}^{N \times (p+1)},
$$
то задача минимизации функции потерь записывается аналогично обычной линейной регрессии:
$$
\mathcal{L}(\mathbf{w}) = \|\mathbf{y} - \Phi \mathbf{w}\|^2.
$$
Нормальное уравнение для нахождения параметров имеет вид:
$$
\Phi^\top \Phi \mathbf{w} = \Phi^\top \mathbf{y},
$$
и аналитическое решение:
$$
\mathbf{w} = (\Phi^\top \Phi)^{-1} \Phi^\top \mathbf{y}.
$$

Замечания, касающиеся линейной регрессии справедливы и в этом случае, однако, теперь модель линейна только по параметрам и нелинейна по признакам. Это позволяет её применять к более сложным зависимостям.

<img src="../../images/pole_regression.jpg" alt="Полиномиальная регрессия" width="600">

Практический пример:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
np.random.seed(42)

N = 100
x = 10 * np.random.rand(N, 1)
noise = np.random.randn(N, 1) * 10
y = 1 + 2 * x + 3 * x ** 2 + noise

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(x, y)
y_lin_pred = lin_reg.predict(x)

poly_degree = 2
poly_transformer = PolynomialFeatures(degree=poly_degree)
X_poly = poly_transformer.fit_transform(x)

poly_reg = LinearRegression()
poly_reg.fit(X_poly, y)
y_poly_pred = poly_reg.predict(X_poly)

In [ ]:
x_grid = np.linspace(x.min(), x.max(), 300).reshape(-1, 1)
y_lin_grid = lin_reg.predict(x_grid)
X_grid_poly = poly_transformer.transform(x_grid)
y_poly_grid = poly_reg.predict(X_grid_poly)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(x, y, color="black", 
            label="Data", alpha=0.7)
plt.plot(x_grid, y_lin_grid, color="blue", linewidth=2, 
         label="Linear Regression")
plt.plot(x_grid, y_poly_grid, color="red", linewidth=2, 
         label="Polynomial Regression degree={}".format(poly_degree))
plt.xlabel("Feature (x)")
plt.ylabel("Target (y)")
plt.title("Linear and Polynomial Regression")
plt.legend()
plt.grid(True)
plt.show()

### Нелинейная регрессия как общий случай

#### Постановка задачи
В более общем случае модель задается как:
$$
y = f(x; \boldsymbol{\theta}) + \varepsilon,
$$
где $ f(x; \boldsymbol{\theta}) $ – произваолная нелинейная функция, зависящая от параметров $ \boldsymbol{\theta} $ (ее вид мы задаем заранее из некоторых предположений), а $ \varepsilon $ – случайная ошибка (шум).

Цель – найти такие параметры $ \boldsymbol{\theta} $, которые минимизируют сумму квадратов ошибок:
$$
\min_{\boldsymbol{\theta}} \sum_{i=1}^{N} \left( y_i - f(x_i; \boldsymbol{\theta}) \right)^2.
$$

Для данной задачи отсутствует аналитическое решение.

Практический пример:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
def nonlinear_model(x, a, b, c, d):
    """
    Модель нелинейной регрессии.

    Аргументы:
        x (np.ndarray): независимая переменная.
        a, b, c, d (float): параметры модели.
    
    Возвращает:
        np.ndarray: значение функции a*sin(b*x) + c*sin(d*x).
    """
    return a * np.sin(b * x) + c * np.sin(d * x)

In [ ]:
np.random.seed(42)
x = np.linspace(0, 10, 200)
a_true, b_true, c_true, d_true = 1.0, 1.0, 0.3, 2.0
true_y = a_true * np.sin(b_true * x) + c_true * np.sin(d_true * x)
noise = np.random.normal(0, 0.2, size=x.shape)
y = true_y + noise

initial_guess = [0.8, 0.8, 0.1, 1.5]
popt, pcov = curve_fit(nonlinear_model, x, y, p0=initial_guess)

print("Parameters of the model and true values:")
print(f"a = {popt[0]:.2f}, true value {a_true}")
print(f"b = {popt[1]:.2f}, true value {b_true}")
print(f"c = {popt[2]:.2f}, true value {c_true}")
print(f"d = {popt[3]:.2f}, true value {d_true}")

In [ ]:
y_fit = nonlinear_model(x, *popt)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(x, y, color="blue", label="Data", alpha=0.6)
plt.plot(x, true_y, 'g--', linewidth=2, label="Ground truth")
plt.plot(x, y_fit, 'r-', linewidth=2, label="Fitted curve")
plt.xlabel("x")
plt.ylabel("y")
plt.title("Nonlinear Regression")
plt.legend()
plt.grid(True)
plt.show()

### Итерационные численные методы

При отсутствии аналитического решения решают задачу оптимизации итерационными (а обычно именно градиентными) методами:
- **Градиентный спуск:** Обновление параметров по правилу:
  $$
  \boldsymbol{\theta}^{k+1} = \boldsymbol{\theta}^{k} - \alpha \nabla_{\boldsymbol{\theta}} \mathcal{L}\left(\boldsymbol{\theta}^{k}\right),
  $$
  где $ \alpha $ – скорость обучения (шаг градиентного спуска).
- **Модфикация метода Ньютона:** Эти методы используют вторые (или аппроксимации вторых) производных функции потерь для ускорения сходимости.

> Подробнее с методами оптимизации первого порядка можно ознакомиться в [учебнике](https://education.yandex.ru/handbook/ml/article/optimizaciya-v-ml) или на одной из следующих лекций.

> Подробнее про оптимизацию второго порядка в [учебнике](https://education.yandex.ru/handbook/ml/article/metody-vtorogo-poryadka)

<img src="../../images/grad_desc.png" alt="Градиентный спуск" width="500">

Практический пример:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
def compute_loss(X, y, weights):
    """
    Вычисляет среднеквадратичную ошибку.

    Аргументы:
        X (np.ndarray): Расширенная матрица признаков (с единичным столбцом).
        y (np.ndarray): Целевая переменная.
        weights (np.ndarray): Вектор параметров [b, w]ᵀ.
    
    Возвращает:
        float: Среднеквадратичная ошибка.
    """
    predictions = X.dot(weights)
    errors = y - predictions
    return np.mean(errors ** 2)

In [ ]:
def gradient_descent(X, y, lr=0.001, num_iters=2000):
    """
    Реализует градиентный спуск для линейной регрессии.

    Аргументы:
        X (np.ndarray): Расширенная матрица признаков.
        y (np.ndarray): Целевая переменная.
        lr (float): Скорость обучения.
        num_iters (int): Количество итераций.

    Возвращает:
        tuple: Найденный вектор параметров и список значений функции потерь.
    """
    N, d = X.shape
    weights = np.zeros((d, 1))
    losses = []

    for _ in tqdm(range(num_iters)):
        predictions = X.dot(weights)
        errors = y - predictions
        gradient = (-2 / N) * X.T.dot(errors)
        weights = weights - lr * gradient

        loss = np.mean(errors ** 2)
        losses.append(loss)

    return weights, losses

In [ ]:
np.random.seed(42)
N = 100
X_data = 10 * np.random.rand(N, 1)
true_b = 5.0
true_w = 2.0
noise = np.random.randn(N, 1)
y_data = true_b + true_w * X_data + noise

X_design = np.hstack((np.ones((N, 1)), X_data))
learning_rate = 0.001
num_iterations = 2000
weights, losses = gradient_descent(X_design, y_data, lr=learning_rate, num_iters=num_iterations)

print("Найденные параметры:")
print(f"b: {weights[0,0]:.4f}, истинное значение {true_b}")
print(f"w: {weights[1,0]:.4f}, истинное значение {true_w}")

plt.figure(figsize=(8, 6))
plt.plot(losses, color="purple")
plt.xlabel("Итерация")
plt.ylabel("Среднеквадратичная ошибка")
plt.title("Сходимость градиентного спуска")
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 6))
plt.scatter(X_data, y_data, color="blue", label="Данные", alpha=0.7)

x_grid = np.linspace(0, 10, 300).reshape(-1, 1)
X_grid_design = np.hstack((np.ones((300, 1)), x_grid))
y_pred = X_grid_design.dot(weights)

plt.plot(x_grid, y_pred, color="red", linewidth=2, label="Линия регрессии")
plt.xlabel("Признак x")
plt.ylabel("Целевая переменная y")
plt.title("Линейная регрессия")
plt.legend()
plt.grid(True)
plt.show()

- Поскольку это учебные примеры демонстрирующие логику модели практически с нуля, важно отметить что используемая реализация [`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) используем **аналитическое** решение с обращение матрицы. В нем используется метод [`numpy.linalg.lstsq`](https://numpy.org/devdocs/reference/generated/numpy.linalg.lstsq.html), однако есть также параметр `solver` поддерживающий ускорения, если матрица невырождена.

- Для второго случая, то есть обучения **методом градиентного спуска**, есть другой класс – [`sklearn.linear_model.SGDRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html).

## Методы регуляризации  
**Регуляризация** – это метод предотвращения переобучения модели путем добавления штрафа за "сложность" модели.

**Переобучение (Overfitting)** – это ситуация, когда модель слишком хорошо запоминает обучающие данные и плохо обобщает новые, в то время как модель должна обладать обобщающей способностью.

<img src="../../images/bias_vs_variance.png" alt="Сложность модели и переобучение" width="500">

<img src="../../images/overfitting.png" alt="Переобучение и недообучение" width="600">

Практический пример:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
np.random.seed(42)
X = np.linspace(0, 10, 20)[:, np.newaxis]
y = np.sin(X).ravel() + np.random.normal(scale=0.3, size=X.shape[0])

In [ ]:
degree = 5
poly = PolynomialFeatures(degree)
X_poly = poly.fit_transform(X)

lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
y_pred_linreg = lin_reg.predict(X_poly)

X_grid = np.linspace(0, 10, 100)[:, np.newaxis]
X_grid_poly = poly.transform(X_grid)
y_grid_linreg = lin_reg.predict(X_grid_poly)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X, y, color='black', label='Data')
plt.plot(X_grid, y_grid_linreg, color='magenta', 
         linewidth=2, label='LinearRegression')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()

### **Lasso-регуляризация (L1)**  
Lasso (Least Absolute Shrinkage and Selection Operator) добавляет штраф за **сумму модулей коэффициентов**:  

$$
\mathcal{L}(w) = MSE + \lambda \sum |w_i|
$$

Приводит к **занулению некоторых коэффициентов**, отбрасывая незначимые признаки. Это важное свойство, может быть использовано для проверки того, какие признаки оказывают наибольшее влияние на целевую переменную

<img src="../../images/l1_reg.png" alt="Lasso" width="500">

Практический пример:

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso(alpha=0.1, max_iter=10000)
lasso.fit(X_poly, y)
y_pred_lasso = lasso.predict(X_poly)
y_grid_lasso = lasso.predict(X_grid_poly)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X, y, color='black', label='Data')
plt.plot(X_grid, y_grid_linreg, color='magenta', 
         linewidth=2, label='LinearRegression')
plt.plot(X_grid, y_grid_lasso, color='red', 
         linewidth=2, label='Lasso')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()

### **Ridge-регуляризация (L2)**  
Ridge добавляет штраф за **сумму квадратов коэффициентов**:  

$$
\mathcal{L}(w) = MSE + \lambda \sum w_i^2
$$

Уменьшает значения коэффициентов, но **не зануляет их**.  

<img src="../../images/l2_reg.png" alt="Ridge" width="500">

Практический пример:

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge = Ridge(alpha=0.1)
ridge.fit(X_poly, y)
y_pred_ridge = ridge.predict(X_poly)
y_grid_ridge  = ridge.predict(X_grid_poly)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X, y, color='black', label='Data')
plt.plot(X_grid, y_grid_linreg, color='magenta', 
         linewidth=2, label='LinearRegression')
plt.plot(X_grid, y_grid_lasso, color='red', 
         linewidth=2, label='Lasso')
plt.plot(X_grid, y_grid_ridge, color='blue', 
         linewidth=2, label='Ridge')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()

### **ElasticNet (L1 + L2)**  
ElasticNet сочетает оба метода:  

$$
\mathcal{L}(w) = MSE + \lambda_1 \sum |w_i| + \lambda_2 \sum w_i^2
$$

Иногда $\lambda_2$ принимают равным $(1-\lambda_1)$

Объединяет свойства **Lasso и Ridge**.  

Практический пример:

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
enet = ElasticNet(alpha=0.1, l1_ratio=0.0, max_iter=10000)
enet.fit(X_poly, y)
y_pred_enet = enet.predict(X_poly)
y_grid_enet = enet.predict(X_grid_poly)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X, y, color='black', label='Data')
plt.plot(X_grid, y_grid_linreg, color='magenta', 
         linewidth=2, label='LinearRegression')
plt.plot(X_grid, y_grid_lasso, color='red', 
         linewidth=2, label='Lasso')
plt.plot(X_grid, y_grid_ridge, color='blue', 
         linewidth=2, label='Ridge')
plt.plot(X_grid, y_grid_enet, color='green', 
         linewidth=2, label='ElasticNet')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()

Использование регуляризации доступно не только для линейных моделей и еще не раз пригодится в будущем.

<div class="alert alert-info">
Вы можете попрактиковаться на конкретных данных, выполнив задания из
<a href="../../Workshops/week02_linear_models.ipynb" target="_blank">ноутбука</a>
</div>